In [ ]:
#Convolutional neural network archtitecture for binary classification

In [10]:
#install all necessary libraries 
#pip install Keras
#pip install scikit-plot
#pip install -U scikit-learn
#pip install tensorflow

In [20]:
#importing all necessary libraries

#general libraries
import os

#deep learning libraries 
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Conv2D, MaxPooling2D
from keras.models import model_from_json
from keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.constraints import maxnorm
from keras.utils import np_utils
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras import optimizers

#data format libraries
import pickle

#visualization libraries
import matplotlib.pyplot as plt
import scikitplot as skplt

#math libraries 
import numpy as np
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, cohen_kappa_score, roc_auc_score, precision_recall_fscore_support
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [1]:
#check tensorflow version
#print(tf.__version__)

In [3]:
#set random seed for purposes of reproducibility
seed = 21

In [2]:
#load data
X_train = pickle.load(open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Classifier Data/final_Train Data/X_train.pickle", "rb"))
y_train = pickle.load(open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Classifier Data/final_Train Data/y_train.pickle", "rb"))

X_test = pickle.load(open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Classifier Data/final_Test Data/X_test.pickle", "rb"))
y_test = pickle.load(open("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Data/Classifier Data/final_Test Data/y_test.pickle", "rb"))

#normalizing data in order to fit classifier dimensions later on
X_train = X_train/255.0
X_test = X_test/255.0



In [2]:
#checking pictures and data type 
print(X_train)
print(type(X_train))

In [3]:
#get shape of data
print('X_train shape:', X_train.shape) # rows, pixel, pixel, depth

In [4]:
#print the label of the image
print('the label is:', y_train[6])

In [5]:
#include batch normalization (normalizing the data after each layer) in order to prevent overfittting
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, 
                                beta_initializer='zeros', gamma_initializer='ones', 
                                moving_mean_initializer='zeros', moving_variance_initializer='ones', 
                                beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, 
                                gamma_constraint=None)

In [30]:
#building the model
#The first thing to do is define the format for the model. Keras has several different formats or blueprints 
#to build models on, but Sequential is the most commonly used, and best applicable one. Therefore, I use a 
#Sequential model. 


model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(X_train.shape[1:])))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Dropout(0.1)) 
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu')) 
model.add(layers.MaxPooling2D((2, 2))) 
model.add(layers.Flatten())
model.add(layers.Dropout(0.4)) 
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid')) 

In [6]:
#look at model summary
model.summary()

In [35]:
#compile the model with binary crossentropy and not sparse_categorical_crossentropy 
#because I aim for a binary classification

model.compile(loss='binary_crossentropy', 
              optimizer=optimizers.RMSprop(learning_rate=1e-4),
              metrics=['acc'])


In [3]:
#use these commands to convert labels to np array
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)


In [7]:
#model summary
print(model.summary())

In [38]:
#include early stopping function to prevent overfitting
#accuracy needs to improve at least 0.01% in 10 epochs in order to keep model running
checkpoint_path= "/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Python/CNNmodel.h5"

earlystop_callback = [
      EarlyStopping(monitor='val_loss', patience=10, mode='min', min_delta=0.0001),
      ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, mode='min')]


In [ ]:
#overfitting might be a problem, therefore I included measures to prevent overfitting
#included options in order to avoid overfitting are:

#batch normalization
#add dropout layers 
#increase the dataset by augmentation
#use batch size as large as possible (64)
#increase the layers in neural network
#use callback functions

In [8]:
#training the model, with 40 iterations by also splitting the data in 70% training data and 30% validation data

np.random.seed(seed) #random seed for reproducability 
history = model.fit(X_train, y_train, validation_split=0.3, batch_size=64, epochs=40, verbose=1, callbacks=earlystop_callback) 

In [9]:
#saving the model
model_json = model.to_json()
with open("model.json", "w") as json_file :
    json_file.write(model_json)

model.save_weights("model12.h5")
print("Saved model to disk")

model.save('CNN12.model')

In [ ]:
#evaluating the model 

In [4]:
#load model
model = tf.keras.models.load_model("/Users/there/Doktorand/Journal Paper/Paper BLM Reactions/Python/CNNmodel.h5")

In [10]:
#look at optimizer
model.optimizer

In [11]:
#look at model weights
model.weights

In [12]:
#printing a graph showing the accuracy changes during the training phase
#print(history.history.keys())
#plt.figure(1)
#plt.plot(history.history['acc'])
#plt.plot(history.history['val_acc'])
#plt.title('model acc')
#plt.ylabel('accuracy')
#plt.xlabel('epoch')
#plt.legend(['train', 'validation'], loc='upper left')

In [13]:
#visualize the loss function of model
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [14]:
#evaluate the model's performance with test data
#this is a dataset the model has never seen before; this is another good check for overfitting 
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [11]:
#predicting values
y_pred = model.predict(X_test)

In [15]:
#round values in order to avoid decimal numbers
y_pred = np.where(y_pred > 0.5, 1,0)
y_pred

In [16]:
#look at rounded values
for i in y_pred:
    print(i)

In [17]:
#confusion matrix as another check
confusion_matrix(y_test, y_pred)

In [18]:
#plot absolute confucion matrix 
skplt.metrics.plot_confusion_matrix(y_test, y_pred)

In [19]:
#plot relative confusion matrix 
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True)

In [21]:
#precision score 
precision_score(y_test, y_pred)

In [20]:
#recall score
recall_score(y_test, y_pred)

In [22]:
#F1 score
f1_score(y_test,y_pred)

In [23]:
#Cohen's Kappa score
cohen_kappa_score(y_test, y_pred)

In [24]:
#precision, recall, fscore, count of each category 
precision_recall_fscore_support(y_test, y_pred)

In [25]:
#ROC Curve value 
roc_auc_score(y_test, y_pred)

In [26]:
#plotting roc curve 
fpr, tpr, thresholds = skplt.metrics.roc_curve(y_test,  y_pred) # false positve rate and true positive rate 
auc = roc_auc_score(y_test, y_pred)
plt.plot(fpr,tpr,label="ROC Curve, AUC="+str(auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label="Random chances")
plt.legend(loc=4)
plt.title("Receiver Operating Characteristic (ROC) Curve")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()
print(auc)

In [27]:
#look at false positive rate
fpr